# Playlist Maker
## Joseph Moore
## Anirud Shrestha

# Load in the data

In [30]:
import kagglehub

# Get lyrics dataset from Kaggle
lyrics_path = kagglehub.dataset_download("carlosgdcj/genius-song-lyrics-with-language-information") + '/song_lyrics.csv'

print("Path to lyrics dataset:", lyrics_path)

# Get playlists dataset from Kaggle
playlists_path = kagglehub.dataset_download("asifsadmine/spotify-playlists-dataset") + '/spotify_dataset.csv'

print("Path to lyrics dataset:", playlists_path)

Path to lyrics dataset: C:\Users\Joseph\.cache\kagglehub\datasets\carlosgdcj\genius-song-lyrics-with-language-information\versions\1/song_lyrics.csv
Path to lyrics dataset: C:\Users\Joseph\.cache\kagglehub\datasets\asifsadmine\spotify-playlists-dataset\versions\1/spotify_dataset.csv


In [31]:
import pandas as pd

# Load in to dataframe
df_lyrics = pd.read_csv(lyrics_path)
df_lyrics.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [32]:
# Load in to dataframe
df_playlists = pd.read_csv(playlists_path, on_bad_lines='skip')
df_playlists.head()

,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010


# Clean the data

In [33]:
import re

def clean_lyrics(input_str):
    # Remove content within square brackets, including brackets
    no_brackets = re.sub(r'\[.*?\]', '', input_str)
    # Remove all newline characters
    cleaned = no_brackets.replace('\n', ' ')
    return cleaned

df_lyrics['lyrics'] = [clean_lyrics(lyrics) for lyrics in df_lyrics['lyrics']]

In [34]:
df_lyrics['lyrics'][0:10]

0     Killa Cam, Killa Cam, Cam Killa Cam, Killa Ca...
1       Yeah, hah, yeah, Roc-A-Fella We invite you ...
2    Maybe cause I'm eatin And these bastards fiend...
3       Ugh, Killa! Baby! Kanye, this that 1970s He...
4     So they ask me "Young boy What you gon' do th...
5     Haha Uh-huh No homo (Young Mula, baby!) I say...
6     No, no, no! I told you, I lived this shit I a...
7     Killa, Dipset Man I spit that pimp talk, you ...
8     Ay yo you wonder who I are I guzzle up at the...
9     Now Lord you know, just how hard I try To liv...
Name: lyrics, dtype: object

In [35]:
# Rename all columns at once
df_playlists.columns = ['user_id', 'artist', 'title', 'playlist_name']

In [36]:
df_merged = pd.merge(df_playlists, df_lyrics, on=['title', 'artist'], how='inner')
df_merged.head()

,user_id,artist,title,playlist_name,tag,year,views,features,lyrics,id,language_cld3,language_ft,language
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010,rock,1977,32827,{},"Oh, it's so funny to be seeing you after so l...",457424,en,en,en
1,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,All Be Okay,HARD ROCK 2010,pop,2021,434,{},I'm gonna break down the walls That keep us f...,1976715,en,en,en
2,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,Bright Side,HARD ROCK 2010,pop,2007,167,{},It's just me and my puppy Together we're so ve...,1846022,en,en,en
3,9cc0cfd4d7d7885102480dd99e7a90d6,Paul McCartney,Dance Tonight,HARD ROCK 2010,rock,2007,1761,{},Everybody gonna dance tonight Everybody gonna...,387833,en,en,en
4,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,Everywhere I Go,HARD ROCK 2010,pop,2010,9454,{},And I'll fall on my knees Tell me how's the wa...,934327,en,en,en


In [37]:
from tqdm.notebook import tqdm
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

def encode_text(batch):
    """
    Encodes a line of text into a vector
    """

    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    vectors = []
    with torch.no_grad():
        outputs = model(**inputs)

    cls_embeddings = outputs.last_hidden_state[:, 0, :]
    vectors.append(cls_embeddings.cpu())

    return torch.cat(vectors, dim=0)

cuda


In [39]:
sampled_df = df_merged.sample(frac = 0.01)

playlist_names = sampled_df['playlist_name'].values
titles = sampled_df['title'].values

pos_vectors = []

batch_size = 64
for i in tqdm(range(0, len(sampled_df), batch_size)):
    playlist_vecs = encode_text(list(playlist_names[i:i+batch_size]))
    title_vecs = encode_text(list(titles[i:i+batch_size]))
    for j in range(len(playlist_vecs)):
        pos_vectors.append(playlist_vecs[j] + title_vecs[j])

print(pos_vectors[0])

  0%|          | 0/746 [00:00<?, ?it/s]

tensor([-5.8882e-01,  3.0964e-01, -2.6503e-01, -1.5311e-01, -1.2726e-01,
        -1.6746e-01,  7.5896e-01,  4.6314e-01, -3.5828e-01, -3.1224e-01,
        -1.4977e-01, -2.5326e-01,  6.3377e-02,  6.0347e-01,  4.2186e-01,
        -9.1413e-02, -4.9978e-01,  8.3615e-01,  5.1270e-01, -4.5080e-01,
        -2.3932e-01, -8.8151e-02, -2.3153e-01, -5.0672e-01, -1.3402e-01,
        -3.0258e-01, -3.8467e-02,  3.0752e-01,  2.4477e-01,  1.4304e-01,
        -1.3337e-02,  3.3582e-01, -1.0189e-01,  1.4331e-01,  4.1627e-01,
        -1.9990e-01,  2.7925e-01, -1.0180e-01,  5.2904e-02,  1.4992e-02,
         2.6171e-01, -1.0284e-01,  2.1965e-01,  3.5037e-02,  6.4556e-02,
        -4.7546e-01, -3.5482e+00, -1.0531e-01, -5.8054e-01, -2.2896e-01,
         3.7046e-01,  3.1101e-02,  8.2263e-01,  5.4989e-01, -3.8267e-02,
         7.6973e-01, -3.5366e-01,  1.2927e+00,  4.6081e-01,  3.9018e-01,
        -1.0604e-01, -2.4857e-03, -2.3574e-01,  9.9829e-02, -7.8942e-03,
         4.0700e-01,  6.9218e-02,  3.5487e-01, -6.2

In [53]:
def get_negative_examples(df, current_playlist, n=3, random_state=42):
    """
    Returns n negative examples (rows) where playlist_name != current_playlist
    """
    samples = df.sample(n=n)
    while (current_playlist in samples['playlist_name']):
        samples = df.sample(n=n)
    return samples
    #negative_df = df[df['playlist_name'] != current_playlist]
    #return negative_df.sample(n=n, random_state=random_state)

In [54]:
neg_sample_titles: list[str] = []
neg_sample_playlists = []


for i in tqdm(range(len(sampled_df))):
    row = sampled_df.iloc[i]
    current_playlist = row['playlist_name']
    samples = get_negative_examples(sampled_df, current_playlist)

    for title in samples['title']:
        neg_sample_titles.append(title)
        neg_sample_playlists.append(current_playlist)

  0%|          | 0/47705 [00:00<?, ?it/s]

In [55]:
neg_vectors = []    
for i in tqdm(range(0, len(neg_sample_titles), batch_size)):
    playlist_vec = encode_text(list(neg_sample_playlists[i:i+batch_size]))
    title_vec = encode_text(list(neg_sample_titles[i:i+batch_size]))
    for j in range(len(playlist_vecs)):
        neg_vectors.append(playlist_vecs[j] + title_vecs[j])

  0%|          | 0/2237 [00:00<?, ?it/s]

In [ ]:
X = pos_vectors + neg_vectors
y = ([1] * len(pos_vectors)) + ([0] * len(neg_vectors))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

C:\Users\Joseph\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9986007912766574


## Cosine Similarity way

In [15]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np

# Download tokenizer if not done already
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
# Deduplicate for training
unique_playlists = df_merged['playlist_name'].dropna().drop_duplicates()

In [28]:
# Tokenize unique playlist titles
tokenized_playlists = [word_tokenize(title.lower()) for title in unique_playlists]

In [30]:
w2v = Word2Vec(sentences=tokenized_playlists, vector_size=100, min_count=1, window=5, workers=4)

In [32]:
def get_avg_vector(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)
    

In [35]:
query = "Chill songs"
query_tokens = word_tokenize(query.lower())
query_vec = get_avg_vector(query_tokens, w2v)
query_vec

array([-0.40211633,  0.49770546, -0.06018797,  0.47975257,  0.4409672 ,
       -1.7178718 ,  0.30247933,  1.752794  , -1.0481502 , -0.57120657,
       -0.26584294, -1.9541409 , -0.1700099 ,  0.6309916 ,  0.30916572,
       -0.8280403 ,  0.6873963 , -0.20991166, -0.10473429, -1.6494293 ,
        1.2099946 ,  0.16711217,  0.6284453 , -0.748235  ,  0.47427645,
       -0.01936241, -0.97873425,  0.0394406 , -0.23651612, -0.28884754,
        0.51354647,  0.08268362,  0.40246615, -0.89566964, -0.4600961 ,
        1.4260952 ,  0.4365483 ,  0.24312666, -0.670693  , -0.46866864,
        0.1010094 , -0.93168795, -0.01901323,  0.12683307, -0.17713012,
       -0.04141244, -0.5597807 ,  0.5631971 ,  0.2382963 ,  0.85512185,
        0.3523497 , -0.9190556 , -0.46670175, -0.27154177,  0.59139013,
        0.1684408 ,  0.9520826 , -0.13712955, -0.69848317,  0.48197204,
       -0.6475824 , -0.26139048,  0.62170494,  0.04408996, -0.4163286 ,
        1.1623037 ,  0.43397638,  1.4954872 , -0.5548661 ,  0.79

In [49]:
tokenized_all = [
    word_tokenize(str(title).lower()) if pd.notna(title) else []
    for title in df_merged['playlist_name']
]

playlist_vecs = [get_avg_vector(tokens, w2v) for tokens in tokenized_all]

In [50]:

def get_cosine_similarity_song(query):
    query_tokens = word_tokenize(query.lower())
    query_vec = get_avg_vector(query_tokens, w2v)
    
    # Cosine similarity
    similarities = cosine_similarity([query_vec], playlist_vecs).flatten()

    # Sort with argsort
    sorted_indices = similarities.argsort()[::-1]
    df_sorted = df_merged.iloc[sorted_indices].copy()
    df_sorted['similarity'] = similarities[sorted_indices]

    return df_sorted[:5]

In [51]:
get_cosine_similarity_song("Love story")

,user_id,artist,title,playlist_name,tag,year,views,features,lyrics,id,language_cld3,language_ft,language,similarity
986405,58da90ea2640a507cc4ada6441bbd214,Taylor Swift,Love Story,Love Story,country,2008,978791,{},[Verse 1]\nWe were both young when I first saw...,98867,en,en,en,1.000000
4278373,658159ed7b84bf407b1edd7bf7ce30c8,Hayseed Dixie,Cat Scratch Fever,Mountain Love,pop,2002,55,{},"Well, I don't know where they come from but th...",1702826,en,en,en,0.998227
4278378,658159ed7b84bf407b1edd7bf7ce30c8,Hayseed Dixie,Walk This Way,Mountain Love,pop,2002,190,{},Backstroke lover\nAlways hidin' 'neath the cov...,1255709,en,en,en,0.998227
4278377,658159ed7b84bf407b1edd7bf7ce30c8,Hayseed Dixie,The Perfect Woman,Mountain Love,pop,2002,154,{},You know I like it so you play the game\nYou p...,1420375,en,en,en,0.998227
4278376,658159ed7b84bf407b1edd7bf7ce30c8,Hayseed Dixie,Feel Like Making Love,Mountain Love,pop,2002,37,{},"Baby, when I think about you, I think about lo...",1324261,en,en,en,0.998227


In [52]:
get_cosine_similarity_song("Hard rock")

,user_id,artist,title,playlist_name,tag,year,views,features,lyrics,id,language_cld3,language_ft,language,similarity
617655,6649c6b017eec9ffc91f83d6f386c1f6,Intwine,Cruel Man,Hard Rock,pop,2004,1550,{},"Sick of preaching this to you\nNobody listens,...",1329717,en,en,en,1.0
3585797,e979c447d3747b097b58a83f1fb4e894,Uriah Heep,Hot Persuasion,Hard Rock,pop,1982,102,{},You know you light my candle\nYou know you loa...,914695,en,en,en,1.0
3585799,e979c447d3747b097b58a83f1fb4e894,Godsmack,I Am,Hard Rock,rock,2003,1831,{},[Verse 1]\nI am your spoken truth\nI am the li...,273374,en,en,en,1.0
3585800,e979c447d3747b097b58a83f1fb4e894,Godsmack,I Stand Alone,Hard Rock,rock,2002,34945,{},"[Verse 1]\nI've told you this once before, you...",209485,en,en,en,1.0
3585801,e979c447d3747b097b58a83f1fb4e894,Uriah Heep,I Wanna Be Free,Hard Rock,pop,1971,829,{},[Verse 1]\nAs I wake up every day\nWith no new...,1573185,en,en,en,1.0


In [53]:
get_cosine_similarity_song("Lo-fi chill")

,user_id,artist,title,playlist_name,tag,year,views,features,lyrics,id,language_cld3,language_ft,language,similarity
1017465,a1636d750f67c7e6754165a2f7856860,Dave Matthews Band,Belly Full,Chill DMB,rock,2012,2037,{},"[Verse 1]\nOh, my love\nIf I had my way\nThen ...",186079,en,en,en,0.999739
1017475,a1636d750f67c7e6754165a2f7856860,Dave Matthews Band,Snow Outside,Chill DMB,rock,2012,2805,{},"[Verse 1]\nWell, you are like a secret garden\...",186113,en,en,en,0.999739
1017463,a1636d750f67c7e6754165a2f7856860,Jack Johnson,A Pirate Looks At Forty,Chill DMB,pop,2012,3211,{},"Mother, mother ocean, well I have heard you ca...",1798565,en,en,en,0.999739
1017464,a1636d750f67c7e6754165a2f7856860,Dave Matthews Band,Baby Blue,Chill DMB,rock,2009,5764,{},[Verse 1]\nConfess your kiss\nStill knocks me ...,68711,en,en,en,0.999739
1017466,a1636d750f67c7e6754165a2f7856860,Dave Matthews Band,Big Eyed Fish,Chill DMB,rock,2002,7708,{},[Verse 1]\nLook at this big-eyed fish swimmin'...,187932,en,en,en,0.999739


In [54]:
get_cosine_similarity_song("rap")

,user_id,artist,title,playlist_name,tag,year,views,features,lyrics,id,language_cld3,language_ft,language,similarity
4169870,a5993960605e6586e5dffabcd30c2757,YG,Do It To Ya,Rap,rap,2014,168895,{TeeFlii},"[Intro: YG]\nHey, hey, hey, hey\nMustard on th...",381670,en,en,en,1.0
693664,57004eb5876f347f53d9d5c237e8bde6,D12,Fight Music,rap,rap,2001,244185,{},"[Chorus: Eminem]\nThis kind of music, use it, ...",1890,en,en,en,1.0
693675,57004eb5876f347f53d9d5c237e8bde6,Lil Wayne,Go DJ,rap,rap,2004,156237,"{""Mannie Fresh""}","[Intro: Mannie Fresh]\nYeah, yeah, yeah, yeah,...",250,en,en,en,1.0
693674,57004eb5876f347f53d9d5c237e8bde6,D12,Girls,rap,rap,2001,275,{},[Eminem]\nHey yo dawg\nI got some shit on my m...,7682026,en,en,en,1.0
693673,57004eb5876f347f53d9d5c237e8bde6,Busta Rhymes,Gimme Some More,rap,rap,1998,88091,{},"[Intro]\nYeah, as a shorty, playing in the fro...",9010,en,en,en,1.0
